In [1]:
import tensorflow as tf

IS_TRAIN=True

# leaky relu function
def lrelu(X, leak=0.2):
    f1 = 0.5 * (1 + leak)
    f2 = 0.5 * (1 - leak)
    return f1 * X + f2 * tf.abs(X)

def get_conv_weight(name,kshape,wd=0.0005):
    #with tf.device('/cpu:0'):
    var=tf.get_variable(name,shape=kshape,initializer=tf.contrib.layers.xavier_initializer())
    if wd!=0:
        weight_decay = tf.nn.l2_loss(var)*wd
        tf.add_to_collection('weightdecay_losses', weight_decay)
    return var

def conv3d(name,l_input,kernel_size, strides=[1,1,1,1,1], padding="SAME", with_w=True):
    conv_w=get_conv_weight(name=name+'_conv3d', kshape=kernel_size)
    bias_w=get_conv_weight(name+'_bias',[kernel_size[-1]],0)
    
    conv = tf.nn.bias_add(tf.nn.conv3d(l_input, conv_w, strides=strides, padding=padding), bias_w, name=name)
    
    if with_w:
        return conv, conv_w, bias_w
    else:
        return conv
    
def convS(name,l_input,in_channels,out_channels, strides=[1,1,1,1,1], padding='SAME', with_w=True):
    conv_w=get_conv_weight(name=name+'_conv3d', kshape=[1,3,3,in_channels,out_channels])
    bias_w=get_conv_weight(name+'_bias', [out_channels], 0)
    
    conv = tf.nn.bias_add(tf.nn.conv3d(l_input, conv_w, strides=strides, padding=padding), bias_w, name=name)
    
    if with_w:
        return conv, conv_w, bias_w
    else:
        return conv
    
def convT(name,l_input,in_channels,out_channels, n_t=3, strides=[1,1,1,1,1], padding='SAME', with_w=True):
    conv_w=get_conv_weight(name=name+'_conv3d', kshape=[n_t,1,1,in_channels,out_channels])
    bias_w=get_conv_weight(name+'_bias', [out_channels], 0)
    
    conv = tf.nn.bias_add(tf.nn.conv3d(l_input, conv_w, strides=strides, padding=padding), bias_w, name=name)
    
    if with_w:
        return conv, conv_w, bias_w
    else:
        return conv
    
def deconv3D(name, l_input, out_shape, kernel_size, strides=[1,1,1,1,1], padding='SAME', with_w=True):
    # output shape : (b, temporal depth, w, h, out_ch)
    # filters : output_chnnel size
    # kernel_size : size of kernel [kernel_temporal, filter_w, filter_h, out_ch, in_ch]
    # strides : size of kernel [temporal, spatio_w, spatio_h, out_ch, in_ch]
    # tf.nn.conv3d_transpose(input, filter=get_conv_weight
    deconv_w=get_conv_weight(name=name+'_deconv3d', kshape=kernel_size)
    print(" >>deconv. l_input", l_input)
    print(" >>deconv. deconv_w", deconv_w)
    deconv = tf.nn.conv3d_transpose(l_input, filter=deconv_w, output_shape=out_shape,
                                    strides=strides, padding=padding, name=name+'_deconv')
    
    if with_w:
        return deconv, deconv_w
    else:
        return deconv

/Users/kimsu/py36tf1x/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [11]:

class P3DBottleNeck():
    def __init__(self, l_input, in_channels, channels, out_channels, n_t=3, downsample=False, _id=0, pType="A", activation=True):
        self.X = l_input
        self.in_channels = in_channels
        self.channels = channels
        self.out_channels = out_channels
        self.n_t = n_t
        self.pType = pType
        self.id = _id
        self.downsample = downsample
        if downsample:
            self.strides=[1,1,2,2,1]
        else:
            self.strides=[1,1,1,1,1]
        self.activation=activation
    
    def P3D_A(self, name, x, with_w=True):
        w_list=[]
        b_list=[]
        if with_w:
            x, w, b=convS(name+'_S', x, self.channels, self.channels, with_w=with_w)
            w_list.append(w)
            b_list.append(b)
        else:
            x=convS(name+'_S', x, self.channels, self.channels, with_w=with_w)
        x=tf.layers.batch_normalization(x, training=IS_TRAIN)
        x=tf.nn.relu(x)
        
        if with_w:
            x, w, b=convT(name+'_T', x, self.channels, self.channels, n_t=self.n_t, with_w=with_w)
            w_list.append(w)
            b_list.append(b)
        else:
            x=convT(name+'_T', x, self.channels, self.channels, n_t=self.n_t, with_w=with_w)
        x=tf.layers.batch_normalization(x, training=IS_TRAIN)
        x=tf.nn.relu(x)
        
        if with_w:
            return x, w_list, b_list
        else:
            return x
    
    def P3D_B(self, name, x, with_w=True):
        w_list=[]
        b_list=[]
        if with_w:
            x_s, w, b=convS(name+'_S', x, self.channels, self.channels, with_w=with_w)
            w_list.append(w)
            b_list.append(b)
        else:
            x_s=convS(name+'_S', x, self.channels, self.channels, with_w=with_w)
        x_s=tf.layers.batch_normalization(x_s, training=IS_TRAIN)
        x_s=tf.nn.relu(x_s)
        
        if with_w:
            x_t, w, b=convT(name+'_T', x, self.channels, self.channels, n_t=self.n_t, with_w=with_w)
            w_list.append(w)
            b_list.append(b)
        else:
            x_t=convT(name+'_T', x, self.channels, self.channels, n_t=self.n_t, with_w=with_w)
        x_t=tf.layers.batch_normalization(x_t, training=IS_TRAIN)
        x_t=tf.nn.relu(x_s)
        
        if with_w:
            return x_s + x_t, w_list, b_list
        else:
            return x_s + x_t
    
    def P3D_C(self, name, x, with_w=True):
        w_list=[]
        b_list=[]
        if with_w:
            x_s, w, b=convS(name+'_S', x, self.channels, self.channels, with_w=with_w)
            w_list.append(w)
            b_list.append(b)
        else:
            x_s=convS(name+'_S', x, self.channels, self.channels, with_w=with_w)
        x_s=tf.layers.batch_normalization(x_s, training=IS_TRAIN)
        x_s=tf.nn.relu(x_s)
        
        if with_w:
            x_st, w, b=convT(name+'_T', x_s, self.channels, self.channels, n_t=self.n_t, with_w=with_w)
            w_list.append(w)
            b_list.append(b)
        else:
            x_st=convT(name+'_T', x_s, self.channels, self.channels, n_t=self.n_t, with_w=with_w)
        x_st=tf.layers.batch_normalization(x_st, training=IS_TRAIN)
        x_st=tf.nn.relu(x_st)
        
        if with_w:
            return x_s + x_st, w_list, b_list
        else:
            return x_s + x_st
    
    def build(self, with_w=True):
        residual = self.X
        w_list=[]
        b_list=[]
        
        # 1x1x1 conv : in_channels --> channels
        conv_w=get_conv_weight('conv_1w_{}'.format(self.id), [1, 1, 1, self.in_channels, self.channels])
        bias_w=get_conv_weight('conv_1b_{}'.format(self.id), [self.channels])
        out=tf.nn.conv3d(self.X, conv_w, strides=self.strides, padding='SAME')
        out=tf.nn.bias_add(out, bias_w, name='conv_1_{}'.format(self.id))
        out=tf.layers.batch_normalization(out, training=IS_TRAIN)
        out=tf.nn.relu(out)
        
        w_list.append(conv_w)
        b_list.append(bias_w)
        
        # P3D : channels --> channels
        if self.pType == "A":
            out=self.P3D_A(name="P3D_A_{}".format(self.id), x=out, with_w=with_w)
        elif self.pType == "B":
            out=self.P3D_B(name="P3D_B_{}".format(self.id), x=out, with_w=with_w)
        else:
            out=self.P3D_C(name="P3D_C_{}".format(self.id), x=out, with_w=with_w)
       
        if with_w:
                w_list += out[1]
                b_list += out[2]
                out=out[0]
                
        # Residual
        # 1x1x1 conv : channels --> out_channels
        conv_w=get_conv_weight('conv_2w_{}'.format(self.id), [1, 1, 1, self.channels, self.out_channels])
        bias_w=get_conv_weight('conv_2b_{}'.format(self.id), [self.out_channels])
        out=tf.nn.conv3d(out, conv_w, strides=[1,1,1,1,1], padding='SAME')
        out=tf.nn.bias_add(out, bias_w, name='conv_2_{}'.format(self.id))
        out=tf.layers.batch_normalization(out, training=IS_TRAIN)
        #out=tf.nn.relu(out)
        
        w_list.append(conv_w)
        b_list.append(bias_w)
        
        # down-sampling residual : in_channels --> out_chaneels
        conv_w=get_conv_weight('dw3d_w_{}'.format(self.id), [1, 1, 1, self.in_channels, self.out_channels])
        bias_w=get_conv_weight('dw3d_b_{}'.format(self.id), [self.out_channels])
        residual=tf.nn.conv3d(residual, conv_w, strides=self.strides, padding='SAME')
        residual=tf.nn.bias_add(residual, bias_w, name='dw3d_{}'.format(self.id))
        residual=tf.layers.batch_normalization(residual, training=IS_TRAIN)
        
        w_list.append(conv_w)
        b_list.append(bias_w)
        
        if self.activation:    
            out += residual
            out = tf.nn.relu(out, name="{}_btn_P3D_{}".format(self.id, self.pType))
        else:
            out = tf.add(out, residual, name="{}_btn_P3D_{}".format(self.id, self.pType))
        
        if with_w:
            return out, w_list, b_list
        else:
            return out


class buildP3DBlock():
    def __init__(self, l_input, in_channels, channels, out_channels, iteration, cnt,
                 n_t=3, downsample=False, last_activation=True):
        self.input = l_input
        self.in_ch = in_channels
        self.ch = channels
        self.out_ch = out_channels
        self.iteration = iteration
        self.cnt = cnt
        self.n_t = n_t
        self.ptype = ["A", "B", "C"]
        self.last_activation = last_activation
        self.downsample=downsample
    
    def build(self, with_w=True):
        len_ptype=len(self.ptype)
        
        w_list=[]
        b_list=[]
        
        ptype=self.ptype[self.cnt%len_ptype]
        x = P3DBottleNeck(l_input=self.input,
                          in_channels=self.in_ch, 
                          channels=self.ch, 
                          out_channels=self.out_ch,
                          n_t=self.n_t,
                          _id=self.cnt, 
                          downsample=self.downsample,
                          pType=ptype).build(with_w=with_w)
        if with_w:
            w_list += x[1]
            b_list += x[2]
            x=x[0]
        print(x)
        
        last_iter = self.cnt + self.iteration - 1
        for i in range(self.cnt + 1, self.cnt + self.iteration):
            ptype=self.ptype[i%len_ptype]
            if i < last_iter:
                x = P3DBottleNeck(l_input=x, 
                                  in_channels=self.out_ch, 
                                  channels=self.ch, 
                                  out_channels=self.out_ch, 
                                  n_t=self.n_t,
                                  _id=i, 
                                  pType=ptype).build(with_w=with_w)
            else:
                x = P3DBottleNeck(l_input=x, 
                                  in_channels=self.out_ch, 
                                  channels=self.ch, 
                                  out_channels=self.out_ch, 
                                  n_t=self.n_t,
                                  _id=i, 
                                  pType=ptype,
                                  activation=self.last_activation).build(with_w=with_w)

            if with_w:
                w_list += x[1]
                b_list += x[2]
                x=x[0]
            print(x)
        
        if with_w:
            return x, w_list, b_list
        else:
            return x

In [21]:

class AAE():
    def __init__(self, img_ch=1, n_t=2): 
        '''
          Parameters
             x   : input tensor
             n_t : step size of temporal direction (used in P3D, convT())
        '''
        #self.x = x
        self.n_t = n_t
        self.img_ch = img_ch
        
        #self.input_shape = self.x.get_shape().as_list()
    
    def encoder(self, _x, with_w=True):
        w_list=[]
        b_list=[]
        with tf.variable_scope("Encoder", reuse=tf.AUTO_REUSE):    
            cnt=0
            # _x: [batch, t, h, w, 3]
            # x: [batch, t, h, w, 32]
            conv_w=get_conv_weight('AAE_E_conv1_w', [1, 3, 3, self.img_ch, 32])
            bias_w=get_conv_weight('AAE_E_conv1_b', [32])
            x=tf.nn.conv3d(_x, conv_w, strides=[1,1,1,1,1], padding='SAME')
            x=tf.nn.bias_add(x, bias_w, name='AAE_E_conv1')
            x=tf.layers.batch_normalization(x, training=IS_TRAIN)
            x=tf.nn.relu(x)
            
            w_list.append(conv_w)
            b_list.append(bias_w)
            print(x)

            # x: [batch, t, h, w, 32]
            iteration=3
            x=buildP3DBlock(x, 32, 16, 32, iteration=iteration, cnt=cnt, n_t=self.n_t).build(with_w=with_w)
            cnt += iteration
            if with_w:
                w_list += x[1]
                b_list += x[2]
                x=x[0]
                print("with_w: ", with_w)

            # x: [batch, t/2, h/2, w/2, 32]
            x= tf.nn.max_pool3d(x, [1, 3, 3, 3, 1], strides=[1, self.n_t, 2, 2, 1], padding='SAME')

            # x: [batch, t/2, h/2, w/2, 64]
            iteration=5
            x=buildP3DBlock(x, 32, 16, 64, iteration=iteration, cnt=cnt, n_t=self.n_t).build(with_w=with_w)
            cnt += iteration
            if with_w:
                w_list += x[1]
                b_list += x[2]
                x=x[0]

            # x: [batch, t/4, h/4, w/4, 64]
            x= tf.nn.max_pool3d(x, [1, 3, 1, 1, 1], strides=[1, self.n_t, 2, 2, 1], padding='SAME')

            # x: [batch, t/4, h/4, w/4, 64]
            iteration=5
            x=buildP3DBlock(x, 64, 32, 64, iteration=iteration, cnt=cnt, n_t=self.n_t).build(with_w=with_w)
            cnt += iteration
            if with_w:
                w_list += x[1]
                b_list += x[2]
                x=x[0]

            # x: [batch, t/8, h/4, w/4, 64]
            x= tf.nn.max_pool3d(x, [1, 3, 3, 3, 1], strides=[1, self.n_t, 1, 1, 1], padding='SAME')

            # x: [batch, t/8, h/4, w/4, 128]
            iteration=8
            x=buildP3DBlock(x, 64, 32, 128, iteration=iteration, cnt=cnt, n_t=self.n_t, 
                         last_activation=False).build(with_w=with_w)
            cnt += iteration
            if with_w:
                w_list += x[1]
                b_list += x[2]
                x=x[0]
            
            conv_w=get_conv_weight('AAE_E_conv2_w', [1, 3, 3, 128, 128])
            bias_w=get_conv_weight('AAE_E_conv2_b', [128])
            x=tf.nn.conv3d(x, conv_w, strides=[1,1,1,1,1], padding='SAME')
            x=tf.nn.bias_add(x, bias_w, name='AAE_E_conv2')

            w_list.append(conv_w)
            b_list.append(bias_w)
            
        if with_w:
            return x, w_list, b_list
        else:
            return x
    
    def decoder(self, _x, with_w=True):
        w_list=[]
        b_list=[]
        
        with tf.variable_scope("Decoder", reuse=tf.AUTO_REUSE):
            cnt=21
            # _x: [batch, t/8, h/4, w/4, 128]
            iteration=8
            x=buildP3DBlock(_x, 128, 32, 64, iteration=iteration, cnt=cnt, n_t=self.n_t).build(with_w=with_w)
            cnt += iteration
            if with_w:
                w_list += x[1]
                b_list += x[2]
                x=x[0]

            # x: [batch, t/4, h/4, w/4, 64]
            o_size=x.get_shape().as_list()  # [b, t, h, w, c]
            print(" >> x: ", x)
            #x=deconv3D("AAE_D_deconv1", x, [o_size[0], o_size[1]*self.n_t, o_size[2], o_size[3], o_size[4]], [self.n_t, 3, 3, 64, 64], with_w=with_w)
            x=deconv3D("AAE_D_deconv1", x, [o_size[0], o_size[1]*self.n_t, o_size[2], o_size[3], o_size[4]], [2, 3, 3, 64, 64], with_w=with_w)
            if with_w:
                w_list.append(x[1])
                x=x[0]
                
            # x: [batch, t/4, h/4, w/4, 64]
            iteration=5
            x=buildP3DBlock(x, 64, 32, 64, iteration=iteration, cnt=cnt, n_t=self.n_t).build(with_w=with_w)
            cnt += iteration
            if with_w:
                w_list += x[1]
                b_list += x[2]
                x=x[0]

            # x: [batch, t/2, h/2, w/2, 64]
            o_size=x.get_shape().as_list()  # [b, t, h, w, c]
            x=deconv3D("AAE_D_deconv2", x, [o_size[0], o_size[1]*self.n_t, o_size[2]*2, o_size[3]*2, o_size[4]], [self.n_t, 3, 3, 64, 64], with_w=with_w)
            #print(x)
            if with_w:
                w_list.append(x[1])
                x=x[0]
            
            # x: [batch, t/2, h/2, w/2, 32]
            iteration=5
            x=buildP3DBlock(x, 64, 16, 32, iteration=iteration, cnt=cnt, n_t=self.n_t).build(with_w=with_w)
            cnt += iteration
            if with_w:
                w_list += x[1]
                b_list += x[2]
                x=x[0]

            # x: [batch, t, h, w, 32]
            o_size=x.get_shape().as_list()  # [b, t, h, w, c]
            x=deconv3D("AAE_D_deconv3", x, [o_size[0], o_size[1]*self.n_t, o_size[2]*2, o_size[3]*2, o_size[4]], [self.n_t, 3, 3, 32, 32], with_w=with_w)
            #print(x)
            if with_w:
                w_list.append(x[1])
                x=x[0]
            
            # x: [batch, t, h, w, 32]
            iteration=3
            x=buildP3DBlock(x, 32, 16, 32, iteration=iteration, cnt=cnt, n_t=self.n_t).build(with_w=with_w)
            cnt += iteration
            if with_w:
                w_list += x[1]
                b_list += x[2]
                x=x[0]

            # x: [batch, t, h, w, 3]
            conv_w=get_conv_weight('AAE_D_conv_w', [1, 3, 3, 32, self.img_ch])
            bias_w=get_conv_weight('AAE_D_conv_b', [self.img_ch])
            x=tf.nn.conv3d(x, conv_w, strides=[1,1,1,1,1], padding='SAME')
            x=tf.nn.bias_add(x, bias_w, name='AAE_D_conv')
            x=tf.layers.batch_normalization(x, training=IS_TRAIN)
            x=tf.nn.sigmoid(x, name='AAE_D_x_hat')

            w_list.append(conv_w)
            b_list.append(bias_w)
            
        if with_w:
            return x, w_list, b_list
        else:
            return x
    
    def discriminator(self, z, with_w=True):
        w_list=[]
        b_list=[]
        
        with tf.variable_scope("Discriminator", reuse=tf.AUTO_REUSE):
            # z : [batchsize, t, h, w, c]   ... [none, 1, 40, 40, 128]
            conv_w=get_conv_weight('AAE_Disc_conv1_w', [1, 3, 3, z.get_shape().as_list()[-1], 64])
            bias_w=get_conv_weight('AAE_Disc_conv1_b', [64])
            h1 =tf.nn.conv3d(z, conv_w, strides=[1,1,2,2,1], padding='SAME')
            h1 =tf.nn.bias_add(h1, bias_w, name='AAE_Disc_conv1')
            h1 =tf.layers.batch_normalization(h1, training=IS_TRAIN)
            h1 = tf.nn.relu(h1)
            
            w_list.append(conv_w)
            b_list.append(bias_w)
            
            # [None, 1, 20, 20, 32]
            conv_w=get_conv_weight('AAE_Disc_conv2_w', [1, 1, 1, 64, 32])
            bias_w=get_conv_weight('AAE_Disc_conv2_b', [32])
            h2 =tf.nn.conv3d(h1, conv_w, strides=[1,1,1,1,1], padding='SAME')
            h2 =tf.nn.bias_add(h2, bias_w, name='AAE_Disc_conv2')
            h2 =tf.layers.batch_normalization(h2, training=IS_TRAIN)
            h2 = tf.nn.relu(h2)
            
            w_list.append(conv_w)
            b_list.append(bias_w)
            
            # [None, 1, 20, 20, 1]
            logits = conv3d("AAE_Disc_logits", h2, [1, 1, 1, 32, 1], strides=[1, 1, 1, 1, 1], with_w=with_w)
            
            if with_w:
                w_list.append(logits[1])
                b_list.append(logits[2])
                logits=logits[0]
        
        if with_w:
            return tf.nn.sigmoid(logits), logits, w_list, b_list
        else:
            return tf.nn.sigmoid(logits), logits
    
    def autoencoder(self, _x, z_sample, with_w=True):
        w_list=[]
        b_list=[]
        
        # encoding
        z = self.encoder(_x, with_w=with_w)
        if with_w:
            w_list += z[1]
            b_list += z[2]
            z = z[0]
        print(z)
        # decoding
        y = self.decoder(z)
        if with_w:
            w_list += y[1]
            b_list += y[2]
            y = y[0]
        
        # Reconstruction Loss
        marginal_likelihood = -tf.reduce_mean(tf.reduce_mean(tf.squared_difference(_x, y)))
        
        # GAN Loss
        z_real = z_sample
        z_fake = z
        
        if with_w:
            D_real, D_real_logits, w_list_r, b_list_r = self.discriminator(z_real, with_w=with_w)
            D_fake, D_fake_logits, w_list_f, b_list_f = self.discriminator(z_fake, with_w=with_w)
            w_list += w_list_r
            b_list += b_list_r
            w_list += w_list_f
            b_list += b_list_f
        else:
            D_real, D_real_logits = self.discriminator(z_real, with_w=with_w)
            D_fake, D_fake_logits = self.discriminator(z_fake, with_w=with_w)
        
        # Discriminator Loss
        D_loss_real = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real_logits)))
        D_loss_fake = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake_logits)))
        D_loss = D_loss_real + D_loss_fake
        
        # Generator Loss
        G_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake_logits)))
        
        marginal_likelihood = tf.reduce_mean(marginal_likelihood)
        D_loss = tf.reduce_mean(D_loss)
        G_loss = tf.reduce_mean(G_loss)
        
        if with_w:
            return y, z, -marginal_likelihood, D_loss, G_loss, w_list, b_list
        else:
            return y, z, -marginal_likelihood, D_loss, G_loss
        
    def autoencoder_tmp(self, _x):
        enc = self.encoder(_x)
        
        mu, log_std_sq = tf.split(enc, 2, -1)
        
        eps = tf.random_normal(tf.shape(mu), 0, 1, dtype=tf.float32)
        #z = tf.add(mu, tf.matmul(tf.sqrt(tf.exp(log_std_sq)), eps))
        z = tf.add(mu, tf.sqrt(tf.exp(log_std_sq)) * eps)
        print("Enc: ", enc)
        print("z. : ", z)
        x_hat = self.decoder(z)
        x_hat = tf.clip_by_value(x_hat, 1e-8, 1 - 1e-8)

        # Loss
        marginal_likelihood = tf.reduce_sum(_x * tf.log(x_hat) + (1 - _x) * tf.log(1 - x_hat), -1)
        marginal_likelihood = tf.reduce_mean(marginal_likelihood)

        KL_divergence = 0.5 * tf.reduce_sum(tf.square(mu) + tf.square(sigma) - tf.log(1e-8 + tf.square(sigma)) - 1, 1)

        
        KL_divergence = 0.5 * tf.reduce_sum(tf.square(mu) + tf.exp(1e-8 + log_std_sq) - log_std_sq - 1, -1)
        KL_divergence = tf.reduce_mean(KL_divergence)
        
        ELBO = marginal_likelihood - KL_divergence
        
        loss = -ELBO
        return x_hat, z, enc, loss, -marginal_likelihood, KL_divergence
        
        
    def generate_sample(self, _x):
        z=self.encoder(_x)
        return self.decoder(z)
    
    def build(self, _x):
        1

In [17]:
''' Build Graph'''
# input placeholder
x = tf.placeholder(tf.float32, shape=[None, 8, 224, 224, 1])
z_sample = tf.placeholder(tf.float32, shape=[None, 1, 224//4, 224//4, 1])

In [22]:
''' Network Architecture'''
model = AAE()

In [31]:
# encoding
z = model.encoder(x, with_w=True)

Tensor("Encoder_4/Relu:0", shape=(?, 8, 224, 224, 32), dtype=float32)
Tensor("Encoder_4/0_btn_P3D_A:0", shape=(?, 8, 224, 224, 32), dtype=float32)
Tensor("Encoder_4/1_btn_P3D_B:0", shape=(?, 8, 224, 224, 32), dtype=float32)
Tensor("Encoder_4/2_btn_P3D_C:0", shape=(?, 8, 224, 224, 32), dtype=float32)
with_w:  True
Tensor("Encoder_4/3_btn_P3D_A:0", shape=(?, 4, 112, 112, 64), dtype=float32)
Tensor("Encoder_4/4_btn_P3D_B:0", shape=(?, 4, 112, 112, 64), dtype=float32)
Tensor("Encoder_4/5_btn_P3D_C:0", shape=(?, 4, 112, 112, 64), dtype=float32)
Tensor("Encoder_4/6_btn_P3D_A:0", shape=(?, 4, 112, 112, 64), dtype=float32)
Tensor("Encoder_4/7_btn_P3D_B:0", shape=(?, 4, 112, 112, 64), dtype=float32)
Tensor("Encoder_4/8_btn_P3D_C:0", shape=(?, 2, 56, 56, 64), dtype=float32)
Tensor("Encoder_4/9_btn_P3D_A:0", shape=(?, 2, 56, 56, 64), dtype=float32)
Tensor("Encoder_4/10_btn_P3D_B:0", shape=(?, 2, 56, 56, 64), dtype=float32)
Tensor("Encoder_4/11_btn_P3D_C:0", shape=(?, 2, 56, 56, 64), dtype=float32

In [32]:
z[1][0]

<tf.Variable 'Encoder/AAE_E_conv1_w:0' shape=(1, 3, 3, 1, 32) dtype=float32_ref>

In [26]:
z

<tf.Tensor 'Encoder_3/AAE_E_conv2:0' shape=(?, 1, 56, 56, 128) dtype=float32>

In [28]:
deconv3D("test_deconv2", z, [None, 2, 56*2, 56*2, 128], [2, 3, 3, 128, 128], with_w=False)

 >>deconv. l_input Tensor("Encoder_3/AAE_E_conv2:0", shape=(?, 1, 56, 56, 128), dtype=float32)
 >>deconv. deconv_w <tf.Variable 'test_deconv2_deconv3d:0' shape=(2, 3, 3, 128, 128) dtype=float32_ref>


TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [None, 2, 112, 112, 128]. Consider casting elements to a supported type.

In [41]:
deconv3D2("test_deconv7", z[0], [1, 2, 56*2, 56*2, 128], [2, 3, 3, 128, 128], with_w=False)

 >>deconv. l_input Tensor("Encoder_4/AAE_E_conv2:0", shape=(?, 1, 56, 56, 128), dtype=float32)
 >>deconv. deconv_w <tf.Variable 'test_deconv7_deconv3d:0' shape=(2, 3, 3, 128, 128) dtype=float32_ref>


<tf.Tensor 'test_deconv7_deconv:0' shape=(1, 2, 112, 112, 128) dtype=float32>

In [33]:
def get_conv_weight2(name,kshape,wd=0.0005):
    #with tf.device('/cpu:0'):
    var=tf.get_variable(name, dtype=tf.float32, shape=kshape,initializer=tf.contrib.layers.xavier_initializer())
    if wd!=0:
        weight_decay = tf.nn.l2_loss(var)*wd
        tf.add_to_collection('weightdecay_losses', weight_decay)
    return var

In [34]:
def deconv3D2(name, l_input, out_shape, kernel_size, strides=[1,1,1,1,1], padding='SAME', with_w=True):
    # output shape : (b, temporal depth, w, h, out_ch)
    # filters : output_chnnel size
    # kernel_size : size of kernel [kernel_temporal, filter_w, filter_h, out_ch, in_ch]
    # strides : size of kernel [temporal, spatio_w, spatio_h, out_ch, in_ch]
    # tf.nn.conv3d_transpose(input, filter=get_conv_weight
    deconv_w=get_conv_weight2(name=name+'_deconv3d', kshape=kernel_size)
    print(" >>deconv. l_input", l_input)
    print(" >>deconv. deconv_w", deconv_w)
    deconv = tf.nn.conv3d_transpose(l_input, filter=deconv_w, output_shape=out_shape,
                                    strides=strides, padding=padding, name=name+'_deconv')
    
    if with_w:
        return deconv, deconv_w
    else:
        return deconv

In [23]:
y = model.decoder(z, with_w=False)

Tensor("Decoder_1/21_btn_P3D_A:0", shape=(?, 1, 56, 56, 64), dtype=float32)
Tensor("Decoder_1/22_btn_P3D_B:0", shape=(?, 1, 56, 56, 64), dtype=float32)
Tensor("Decoder_1/23_btn_P3D_C:0", shape=(?, 1, 56, 56, 64), dtype=float32)
Tensor("Decoder_1/24_btn_P3D_A:0", shape=(?, 1, 56, 56, 64), dtype=float32)
Tensor("Decoder_1/25_btn_P3D_B:0", shape=(?, 1, 56, 56, 64), dtype=float32)
Tensor("Decoder_1/26_btn_P3D_C:0", shape=(?, 1, 56, 56, 64), dtype=float32)
Tensor("Decoder_1/27_btn_P3D_A:0", shape=(?, 1, 56, 56, 64), dtype=float32)
Tensor("Decoder_1/28_btn_P3D_B:0", shape=(?, 1, 56, 56, 64), dtype=float32)
 >> x:  Tensor("Decoder_1/28_btn_P3D_B:0", shape=(?, 1, 56, 56, 64), dtype=float32)


ValueError: Trying to share variable Decoder/AAE_D_deconv1_deconv3d, but specified shape (2, 3, 3, 64, 64) and found shape (1, 3, 3, 64, 64).

In [ ]:
tf.AUTO_REUSE

In [6]:
y, z, marginal_likelihood, D_loss, G_loss, w_list, b_list = model.autoencoder(x, z_sample, with_w=True)

Tensor("Encoder/0_btn_P3D_A:0", shape=(?, 8, 224, 224, 32), dtype=float32)
Tensor("Encoder/1_btn_P3D_B:0", shape=(?, 8, 224, 224, 32), dtype=float32)
Tensor("Encoder/2_btn_P3D_C:0", shape=(?, 8, 224, 224, 32), dtype=float32)
Tensor("Encoder/3_btn_P3D_A:0", shape=(?, 4, 112, 112, 64), dtype=float32)
Tensor("Encoder/4_btn_P3D_B:0", shape=(?, 4, 112, 112, 64), dtype=float32)
Tensor("Encoder/5_btn_P3D_C:0", shape=(?, 4, 112, 112, 64), dtype=float32)
Tensor("Encoder/6_btn_P3D_A:0", shape=(?, 4, 112, 112, 64), dtype=float32)
Tensor("Encoder/7_btn_P3D_B:0", shape=(?, 4, 112, 112, 64), dtype=float32)
Tensor("Encoder/8_btn_P3D_C:0", shape=(?, 2, 56, 56, 64), dtype=float32)
Tensor("Encoder/9_btn_P3D_A:0", shape=(?, 2, 56, 56, 64), dtype=float32)
Tensor("Encoder/10_btn_P3D_B:0", shape=(?, 2, 56, 56, 64), dtype=float32)
Tensor("Encoder/11_btn_P3D_C:0", shape=(?, 2, 56, 56, 64), dtype=float32)
Tensor("Encoder/12_btn_P3D_A:0", shape=(?, 2, 56, 56, 64), dtype=float32)
Tensor("Encoder/13_btn_P3D_B:0",

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [None, 2, 56, 56, 64]. Consider casting elements to a supported type.